# Predict the concurrence or the mutual information for whatever two-qubit quantum state you want

In [1]:
"""DNNs will run on GPU if enabled"""
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from numpy import *
from numpy import linalg as la
from scipy import linalg as sa

In [3]:
import keras
from keras import optimizers
from keras import initializers
from keras.models import Sequential
from tensorflow.keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Conv1D, Flatten, Reshape, AveragePooling1D,UpSampling1D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [4]:
"""generate a rank-r quantum state"""

def randomHaarState(dim,rank):
    A = random.normal(0,1,(dim,dim))+1j*random.normal(0,1,(dim,dim))
    q,r = la.qr(A,mode='complete')
    r  = divide(diagonal(r),abs(diagonal(r)))*identity(dim)
    rU = q@r
    B = random.normal(0,1,(dim,rank))+1j*random.normal(0,1,(dim,rank))
    B = B@B.T.conj()
    rho = (identity(dim)+rU)@B@(identity(dim)+rU.T.conj())
    return rho/trace(rho)

"""generate a random Haar pure state"""
def randompure(dim,n):
    rpure = random.normal(0,1,[dim,n]) + 1j*random.normal(0,1,[dim,n])
    rpure = rpure/la.norm(rpure,axis=0)
    rhon = array([dot(rpure[:,[i]],rpure[:,[i]].conjugate().transpose())  for i in range(n)])
#     rhon = reshape(rhon,[n,4])
    return rhon

"""1-qubit Pauli projectors"""
def mubpom():
    p1 = array([1,0])
    p2 = array([0,1])
    mub = zeros([6,1,2])+1j*zeros([6,1,2])
    mub[0] = p1
    mub[1] = p2
    mub[2] = 1/sqrt(2)*(p1+p2)
    mub[3] = 1/sqrt(2)*(p1-p2)
    mub[4] = 1/sqrt(2)*(p1+1j*p2)
    mub[5] = 1/sqrt(2)*(p1-1j*p2)
    mubp = [transpose(mub[i])@conjugate(mub[i]) for i in range(6)]
    return mubp

def mubpom1():
    p1 = array([1,0])
    p2 = array([0,1])
    mub = zeros([4,1,2])+1j*zeros([4,1,2])
    mub[0] = p1
    mub[1] = p2
    mub[2] = 1/sqrt(2)*(p1+p2)
    mub[3] = 1/sqrt(2)*(p1+1j*p2)
    mubp = [transpose(mub[i])@conjugate(mub[i]) for i in range(4)]
    return mubp

def blochFromRho(rho,A):
    l = shape(A)[0]
    return array([ real(trace(rho@A[n])) for n in range(l) ])

"""get probabilities from quantum state rho0 and POVM"""
def probdists(rho0,povm):
    l = shape(povm)[0]
    probtrue = array([real(trace(rho0@povm[i])) for i in range(l)])
    probtrue = probtrue/sum(probtrue)
    return probtrue

def herbasis(dim):
    pom1 = zeros([1,dim,dim])+1j*zeros([1,dim,dim])
    pom1[0] = identity(dim)
    arrays = [dot(transpose(pom1[0][[i]]),pom1[0][[i]]) for i in range(dim-1)]
    pom = stack(arrays,axis=0)
    her = concatenate((pom1,pom),axis=0)
    arrays = [dot(transpose(her[0][[i]]),her[0][[j]])+dot(transpose(her[0][[j]]),her[0][[i]]) for i in range(dim) for j in range(i+1,dim)]
    pom = stack(arrays,axis=0)
    her = concatenate((her,pom),axis=0)
    arrays = [-1j*dot(transpose(her[0][[i]]),her[0][[j]])+1j*dot(transpose(her[0][[j]]),her[0][[i]]) for i in range(dim) for j in range(i+1,dim)]
    pom = stack(arrays,axis=0)
    pom = concatenate((her,pom),axis=0)
    return pom

def gellmann(Q,dim):
    q = zeros([dim**2,dim,dim])+1j*zeros([dim**2,dim,dim])
    for i in range(dim**2):
        v = Q[i]
        for j in range(0,i):
            v = v-trace(v@q[j])*q[j]
        q[i] = v/sqrt(trace(v@v))
    return q

def pauli():
    s = zeros([3,2,2]) +1j*zeros([3,2,2])
    s[0] = array([[1, 0],[0, -1]])
    s[1] = array([[0, 1],[ 1, 0]])
    s[2] = array([[0, -1j],[1j, 0]])
    return s

"""concurrence"""
def conc(A):
    s = pauli()
    s2 = kron(s[2],s[2])
    At = (s2@conjugate(A))@s2
    As = sa.sqrtm(A)
    R = sa.sqrtm((As@At)@As)
    eigval = real(sort(la.eig(R)[0])[::-1])
    return max(0,eigval[0]-(eigval[1]+eigval[2]+eigval[3]))

def parTrB(rho):
    rhoN = zeros([2,2])+1j*zeros([2,2])
    rhoN[0,0]=rho[0,0]+rho[1,1]
    rhoN[0,1]=rho[0,2]+rho[1,3]
    rhoN[1,0]=rho[2,0]+rho[3,1]
    rhoN[1,1]=rho[2,2]+rho[3,3]
    return rhoN

def parTrA(rho):
    rhoN = zeros([2,2])+1j*zeros([2,2])
    rhoN[0,0]=rho[0,0]+rho[2,2]
    rhoN[0,1]=rho[0,1]+rho[2,3]
    rhoN[1,0]=rho[1,0]+rho[3,2]
    rhoN[1,1]=rho[1,1]+rho[3,3]
    return rhoN

"""two-qubit mutual information"""
def mutInf2q(rho):
    rA = parTrB(rho)
    rB = parTrA(rho)
    return real( 1/2*(-trace(rA@myLog(rA))-trace(rB@myLog(rB)) + trace(rho@myLog(rho))) )

def myLog(M):
    U,S,VT = la.svd(M)
    for n in range(shape(S)[0]):
        if S[n]<10**(-14):
            S[n] = 1
    D = diag(log2(S))
    return (U@D)@VT

"""regularized MaxLik algorithm"""
def MLalg(data, rhoinit,povm,epsilon,stop):
    rho = rhoinit
    trdist = 1
    k = 0
    pomS = zeros([4,4], complex)
    for n in range(shape(povm)[0]): pomS = pomS + povm[n]
    Gop = sa.pinv(sa.fractional_matrix_power(pomS,1/2))
    while trdist > epsilon and k<stop:
        R = zeros([4,4], complex)
        prob = probdists(rho,povm)
        for i in range(shape(povm)[0]):
            if data[i]>10**(-10):
                R += data[i]/prob[i]*povm[i]
        rhonew = dot(dot(Gop,dot(R,rho)),dot(R,Gop))
        rhonew = rhonew/trace(rhonew)
        trdist = trace(dot((rho-rhonew),(rho-rhonew)))
        rho = rhonew
        k+=1
    return rho


In [5]:
"""define the basis in the n-dimensional Hilbert space"""
dim = 4
noStates = 1
nProj = 36
Q = herbasis(dim)
GAll = gellmann(Q,dim)*sqrt(dim)
G = GAll[1::]

In [6]:
"""define Pauli projectors for two-qubit system"""
mub = mubpom()
mub2 = array([kron(mub[i],mub[j])/9 for i in range(6) for j in range(6) ])

blochFromPOMs = array([ blochFromRho(mub2[m],GAll) for m in range(nProj) ])

In [7]:
"""generate a random pure two-qubit state"""
rhoT = randompure(dim,noStates);

# p = 0.8
# rhoT = [p*kron(randompure(2,1)[0],randompure(2,1)[0])+(1-p)/4*identity(dim)]



"""add some noise if you want"""
p = 1
rhoT = p*rhoT + (1-p)/4*identity(dim)

"""generate a random rank-r state with respect to the Bures measure"""
# rank = 1
# rhoT = randomHaarState(dim,dim)

"""calculate the full probability distribution"""
probList = probdists(rhoT[0],mub2)

"""check the true value of the concurrence and the mutual information"""
print(conc(rhoT[0]))
print(mutInf2q(rhoT[0]))
print(trace(rhoT[0]@rhoT[0]))

0.7006611682219508
0.5926949242315412
(0.9999999999999997+0j)


# Measurement-specific strategy

In [8]:
"""load trained measurement-specific DNNs"""
nProjAll = 19
pom = array(range(1,nProjAll))

"""Each model is trained for the specific set of projector,
we trained DNNs for five different sets, load one of them, nR = 0,1,2,3,4"""
nR = 0

"""choose the number of measurement settings, from 2 to 36, only even numbers"""
nMeasSettings = 18

file1 = load("spec_pauli_projections/PauliR"+str(nR)+str(nMeasSettings)+".npz")
mub3 = file1['mub3']

"""definition of the vocabulary of measurement-specific models"""
dictModelSpecificConcurrence={}
for k in pom:
    dictModelSpecificConcurrence[str(k)]="spec_net_conc_R"+str(nR)+"/bestModelPauliConcProjR"+str(nR)+"v7"+str(2*k)
    
dictModelSpecificMutualInfo={}
for k in pom:
    dictModelSpecificMutualInfo[str(k)]="spec_net_MI_R0/bestModelPauliMIProjR0v6"+str(2*k)    


In [9]:
"""get the data"""
probListSpec = reshape(probdists(rhoT[0],mub3),(1,nMeasSettings))

In [10]:
"""load models"""
modelSpecificConcurrence = load_model(dictModelSpecificConcurrence[str(int(nMeasSettings/2))]+'.h5')

modelSpecificMutualInfo = load_model(dictModelSpecificMutualInfo[str(int(nMeasSettings/2))]+'.h5')

In [11]:
"""estimation of the concurrence"""
modelSpecificPredictionConcurrence = modelSpecificConcurrence.predict(probListSpec)

"""estimation of the mutual info."""
modelSpecificPredictionMutualInfo = modelSpecificMutualInfo.predict(probListSpec)

In [12]:
"""MaxLik predictions"""
rhoML = MLalg(probListSpec[0],identity(4)/4,mub3,10**(-14),100000)

mlPredictedConcurrenceSpecificProjs = conc(rhoML)

mlPredictedMutualInfoSpecificProjs = mutInf2q(rhoML)

In [13]:
print("true value of the concurrence:", conc(rhoT[0]))
print("measurement-specific DNN prediction of the concurrence based on "+str(nMeasSettings)+" projections:", modelSpecificPredictionConcurrence[0,0])
print("MaxLik prediction of the concurrence:", mlPredictedConcurrenceSpecificProjs)

true value of the concurrence: 0.7006611682219508
measurement-specific DNN prediction of the concurrence based on 18 projections: 0.69630015
MaxLik prediction of the concurrence: 0.4739079266508104


In [14]:
print("true value of the mutual info.:", mutInf2q(rhoT[0]))
print("measurement-specific DNN prediction of the mutual info. based on "+str(nMeasSettings)+" projections:", modelSpecificPredictionMutualInfo[0,0])
print("MaxLik prediction of the mutual info.:", mlPredictedMutualInfoSpecificProjs)

true value of the mutual info.: 0.5926949242315412
measurement-specific DNN prediction of the mutual info. based on 18 projections: 0.5850143
MaxLik prediction of the mutual info.: 0.3053876358165032


# Measurement-independent DNN strategy

In [15]:
"""load measurement-independent DNN that predicts concurrence"""
modelConcurrence = load_model('modelConcurrence.h5',compile = False)
modelMutualInfo = load_model('modelMI.h5',compile = False)

## informationally complete data

In [16]:
"""data preparation for the input layer of the measurement-independent DNN"""
x_test = zeros([int(noStates),17*nProj,1])

for i in range(nProj):
    x_test[0,i*17:i*17+16,0] = blochFromPOMs[i]
    x_test[0,16*(i+1)+i,0] = probList[i]

modelPredictionConcurrence = modelConcurrence.predict(x = x_test)
modelPredictionMutualInfo = modelMutualInfo.predict(x = x_test)

In [17]:
"""MaxLik prediction"""
rhoML = MLalg(probList,identity(4)/4,mub2,10**(-14),100000)
mlPredictedConcurrenceComplete = conc(rhoML)
mlPredictedMutualInfoComplete = mutInf2q(rhoML)

In [18]:
print("true value of the concurrence:", conc(rhoT[0]))
print("measurement-independent DNN prediction of the concurrence:", modelPredictionConcurrence[0,0])
print("MaxLik prediction of the concurrence:", mlPredictedConcurrenceComplete)

true value of the concurrence: 0.7006611682219508
measurement-independent DNN prediction of the concurrence: 0.6901124
MaxLik prediction of the concurrence: 0.7003568144602577


In [19]:
print("true value of the mutual info.:", mutInf2q(rhoT[0]))
print("measurement-independent DNN prediction of the mutual info.:", modelPredictionMutualInfo[0,0])
print("MaxLik prediction of the mutual info.:", mlPredictedMutualInfoComplete)

true value of the mutual info.: 0.5926949242315412
measurement-independent DNN prediction of the mutual info.: 0.58023375
MaxLik prediction of the mutual info.: 0.5916158897423192


## Informationaly incomplete data

In [20]:
"""pick at random, which tuples of projections will be measured to estimate the concurrence"""
"""nExc -> number of excluded projectors"""
nExc = 18

pomS = ones((36),dtype=int)
mub2Exc = mub2
probListExc = probList
"""pick at random which projectors will be excluded and replace respective probabilities
and measurement describtion with zeros"""
myList=list(range(36))
random.shuffle(myList)
for i in myList[0:nExc]:
    mub2Exc[i] = zeros([4,4],complex)
    probListExc[i] = 0

blochFromPOMsExc = array([ blochFromRho(mub2Exc[m],GAll) for m in range(nProj) ])

"""We always normalize data before feeding them to the networks"""
probListExcP = probListExc/sum(probListExc)

x_test_exc = zeros([int(noStates),17*nProj,1])

for i in range(nProj):
    x_test_exc[0,i*17:i*17+16,0] = blochFromPOMsExc[i]
    x_test_exc[0,16*(i+1)+i,0] = probListExcP[i]
    
modelPredictionConcurrenceIncomplete = modelConcurrence.predict(x = x_test_exc)
modelPredictionMutualInfoIncomplete = modelMutualInfo.predict(x = x_test_exc)

In [21]:
rhoML = MLalg(probListExcP,identity(4)/4,mub2Exc,10**(-14),100000)
mlPredictedConcurrenceIncomplete = conc(rhoML)
mlPredictedMutualInfoInComplete = mutInf2q(rhoML)

In [22]:
print("true value of the concurrence:", conc(rhoT[0]))
print("measurement-independent DNN prediction of the concurrence with "+str(nExc)+" projections missing :", modelPredictionConcurrenceIncomplete[0,0])
print("MaxLik prediction of the concurrence with "+str(nExc)+" projections missing :", mlPredictedConcurrenceIncomplete)

true value of the concurrence: 0.7006611682219508
measurement-independent DNN prediction of the concurrence with 18 projections missing : 0.28528163
MaxLik prediction of the concurrence with 18 projections missing : 0.16190822320468412


In [23]:
print("true value of the mutual info.:", mutInf2q(rhoT[0]))
print("measurement-independent DNN prediction of the mutual info. with "+str(nExc)+" projections missing :", modelPredictionMutualInfoIncomplete[0,0])
print("MaxLik prediction of the mutual info. with "+str(nExc)+" projections missing :", mlPredictedMutualInfoInComplete)

true value of the mutual info.: 0.5926949242315412
measurement-independent DNN prediction of the mutual info. with 18 projections missing : 0.3156768
MaxLik prediction of the mutual info. with 18 projections missing : 0.2760453323447879


In [24]:
# pp = ["h","v","d","a","r","l"]
# dictProjs = {}
# for i in range(6):
#     for j in range(6):
#         dictProjs[pp[i]+pp[j]] = kron(mub[i],conjugate(mub[j]))
        
# mub1 = mubpom1()

# pp1 = ["h","v","d","r",]

# sp1 = array([pp1[i]+pp1[j] for i in range(4) for j in range(4)])

In [442]:
# specProjs = array([dictProjs['hh'],dictProjs['hv'],dictProjs['vh'],dictProjs['vv'],dictProjs['rh'],dictProjs['rv'],
#           dictProjs['dv'],dictProjs['dh'],dictProjs['dr'],dictProjs['dd'],dictProjs['rd'],
#            dictProjs['hd'],dictProjs['vd'],dictProjs['vl'],
#           dictProjs['hl'],dictProjs['rl']])

# specProjs1P = array([sp1[i] for i in range(16)])
# specProjs1 = array([dictProjs[i] for i in specProjs1P])